1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)


In [143]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

import matplotlib.pyplot as plt

%matplotlib inline

In [144]:
df = pd.read_csv("churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [145]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=26)

In [146]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [147]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [148]:
scaler = MinMaxScaler()

In [149]:
# scaler.fit(np.vstack((X_train[continuous_columns], X_test[continuous_columns])))

In [150]:
final_transformers = list()
# scaler = MinMaxScaler()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col))
            ])
    final_transformers.append((cont_col, cont_transformer))

In [151]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

In [152]:
pipelineRF = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 26))
    
])
pipelineRF.fit(X_train, y_train)
preds = pipelineRF.predict_proba(X_test)[:, 1]

In [153]:
precisionRF, recallRF, thresholdsRF = precision_recall_curve(y_test, preds)

fscoreRF = (2 * precisionRF * recallRF) / (precisionRF + recallRF)
ix = np.argmax(fscoreRF)
print(f'Best Threshold={thresholdsRF[ix]}, F-Score={fscoreRF[ix]:.3f}, Precision={precisionRF[ix]:.3f}, Recall={recallRF[ix]:.3f}')

Best Threshold=0.35, F-Score=0.630, Precision=0.610, Recall=0.651


In [154]:
pipelineGB = Pipeline([
    ('features',feats),
    ('classifier', GradientBoostingClassifier(random_state = 26)),
])
pipelineGB.fit(X_train, y_train)
preds = pipelineGB.predict_proba(X_test)[:, 1]

In [155]:
precisionGB, recallGB, thresholdsGB = precision_recall_curve(y_test, preds)

fscoreGB = (2 * precisionGB * recallGB) / (precisionGB + recallGB)
ix = np.argmax(fscoreGB)
print(f'Best Threshold={thresholdsGB[ix]}, F-Score={fscoreGB[ix]:.3f}, Precision={precisionGB[ix]:.3f}, Recall={recallGB[ix]:.3f}')

Best Threshold=0.2923309979838839, F-Score=0.645, Precision=0.607, Recall=0.687


In [194]:
pipelineLR = Pipeline([
    ('features',feats),
    ('scaler', scaler),
    ('classifier', LogisticRegression(random_state = 26)),
])
pipelineLR.fit(X_train, y_train)
preds = pipelineLR.predict_proba(X_test)[:, 1]

In [195]:
precisionLR, recallLR, thresholdsLR = precision_recall_curve(y_test, preds)

fscoreLR = (2 * precisionLR * recallLR) / (precisionLR + recallLR)
fscoreLR = np.nan_to_num(fscoreLR)
ix = np.argmax(fscoreLR)
print(f'Best Threshold={thresholdsLR[ix]}, F-Score={fscoreLR[ix]:.3f}, Precision={precisionLR[ix]:.3f}, Recall={recallLR[ix]:.3f}')

Best Threshold=0.19827086071182923, F-Score=0.480, Precision=0.357, Recall=0.732


C:\Users\ASUS\AppData\Local\Temp\ipykernel_1900\1728934764.py:3: RuntimeWarning: invalid value encountered in true_divide
  fscoreLR = (2 * precisionLR * recallLR) / (precisionLR + recallLR)


2.Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)

*По метрикам сильно отстает LogReg, лучшая модель - GradBoost. Тут лучше смотреть на fscore. С одной стороны нам важнее полнота: лучше мы позвоним клиентам, которые не собираются уходить, но точно охватим всех, кого надо переубедить. С другой стороны, мы можем лишний раз побеспокоить тех, кто не собирался уходить, и они передумают.<br>
Но если рассуждать с экономической точки зрения, наверное нам важнее точность, чтобы не тратить деньги на тех, кто и так остаётся.*

3.Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты.

In [221]:
pipelineGB.fit(X_train, y_train)
preds = pipelineGB.predict_proba(X_test)[:, 1]

precisionGB, recallGB, thresholdsGB = precision_recall_curve(y_test, preds)

fscoreGB = (2 * precisionGB * recallGB) / (precisionGB + recallGB)
ix = np.argmax(fscoreGB)
print(f'Best Threshold={thresholdsGB[ix]}, F-Score={fscoreGB[ix]:.3f}, Precision={precisionGB[ix]:.3f}, Recall={recallGB[ix]:.3f}')

Best Threshold=0.2923309979838839, F-Score=0.645, Precision=0.607, Recall=0.687


In [222]:
cnf_matrix = confusion_matrix(y_test, preds>thresholdsLR[ix])
cnf_matrix

array([[1805,  213],
       [ 152,  330]], dtype=int64)

In [223]:
savings = 2*cnf_matrix[1,1]-cnf_matrix[0,1]-cnf_matrix[1,1]
savings

117

По предсказаниям модели выгода составит 117$.<br>
Попробуем перебрать отсечки вручную:

In [224]:
thresh = np.linspace(0,1,100)
savs = []
for i in thresh:
    cnf_matrix = confusion_matrix(y_test, preds>i)
    savs.append(2*cnf_matrix[1,1]-cnf_matrix[0,1]-cnf_matrix[1,1])
ix = np.argmax(savs)
best_thr = thresh[ix]
tot_sav = savs[ix]
print(f'Best threshold: {best_thr:.4f}, savings: {tot_sav}')

Best threshold: 0.5859, savings: 182


На самом деле, лучшая отсечка с экономической точки зрения выше и выгода при ней составит аж 182$


4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3
5. (опционально) Еще раз провести оценку экономической эффективности